#### Урок 3. Парсинг HTML. BS, SQLAlchemy

1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД
2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы
3*)Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [ ]:
# Установка БД
"""
docker run -d -p 127.0.0.1:27017:27017 --name mongo4parsing mongo
docker ps -a
docker start f48a0957b452
docker exec -it f48a0957b452 mongo

"""

In [119]:
from pymongo import MongoClient
#from pprint import pprint
import pandas as pd
import numpy as np
from pprint import pprint

client = MongoClient('mongodb://127.0.0.1:27017')
vacancy_pd = pd.read_pickle('vacancy_pd.pkl')

In [120]:
vacancy_pd
type(vacancy_pd["min_salary"][3])

list

In [121]:
def db_vacancy_writer(df):
    client.db.docs.drop()
    vacancy_db = client['vacancy']
    docs = vacancy_db.docs
    result = docs.insert_many(df.to_dict(orient='records'))
    return result.inserted_ids

In [122]:
db_vacancy_writer(vacancy_pd)


[ObjectId('5db40f66aba5db6b7932ab3b'),
 ObjectId('5db40f66aba5db6b7932ab3c'),
 ObjectId('5db40f66aba5db6b7932ab3d'),
 ObjectId('5db40f66aba5db6b7932ab3e'),
 ObjectId('5db40f66aba5db6b7932ab3f'),
 ObjectId('5db40f66aba5db6b7932ab40'),
 ObjectId('5db40f66aba5db6b7932ab41'),
 ObjectId('5db40f66aba5db6b7932ab42'),
 ObjectId('5db40f66aba5db6b7932ab43'),
 ObjectId('5db40f66aba5db6b7932ab44'),
 ObjectId('5db40f66aba5db6b7932ab45'),
 ObjectId('5db40f66aba5db6b7932ab46'),
 ObjectId('5db40f66aba5db6b7932ab47'),
 ObjectId('5db40f66aba5db6b7932ab48'),
 ObjectId('5db40f66aba5db6b7932ab49'),
 ObjectId('5db40f66aba5db6b7932ab4a'),
 ObjectId('5db40f66aba5db6b7932ab4b'),
 ObjectId('5db40f66aba5db6b7932ab4c'),
 ObjectId('5db40f66aba5db6b7932ab4d'),
 ObjectId('5db40f66aba5db6b7932ab4e'),
 ObjectId('5db40f66aba5db6b7932ab4f'),
 ObjectId('5db40f66aba5db6b7932ab50'),
 ObjectId('5db40f66aba5db6b7932ab51'),
 ObjectId('5db40f66aba5db6b7932ab52'),
 ObjectId('5db40f66aba5db6b7932ab53'),
 ObjectId('5db40f66aba5db

In [123]:
def db_vacancy_search(salary):
    vacancy_db = client['vacancy']
    docs = vacancy_db.docs
    for x in docs.find({},{ "min_salary": {'$gt':salary} }):
        print(docs['vacancy'])

In [125]:
db_vacancy_search(100000)

OperationFailure: Unsupported projection option: min_salary: { $gt: 100000 }

In [118]:
salary = 100000
objects = docs.find({},{ "min_salary": salary }).sort("min_salary")

n = 0
for obj in objects:


    pprint(obj)    
pprint(n)        

{'_id': ObjectId('5db40a80aba5db6b7932aa99'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aa9a'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aa9b'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aa9d'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aa9e'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aa9f'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aaa0'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aaa1'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aaa2'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aaa3'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aaa4'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aaa5'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aaa6'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aaa7'), 'min_salary': [nan]}
{'_id': ObjectId('5db40a80aba5db6b7932aaa8'), 'min_salary': [n

In [ ]:
### пример

import pandas as pd
from pymongo import MongoClient


def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df